In [ ]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 35.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [ ]:
!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip cornell_movie_dialogs_corpus.zip


--2024-07-14 10:23:27--  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.53
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.53|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: ‘cornell_movie_dialogs_corpus.zip’

cornell_movie_dialo 100%[===================>]   9.46M  4.65MB/s    in 2.0s    

2024-07-14 10:23:30 (4.65 MB/s) - ‘cornell_movie_dialogs_corpus.zip’ saved [9916637/9916637]

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-

In [ ]:
import os
import pandas as pd

def load_movie_lines(file_name):
    lines = open(file_name, encoding='iso-8859-1').read().split('\n')
    id2line = {}
    for line in lines:
        _line = line.split(' +++$+++ ')
        if len(_line) == 5:
            id2line[_line[0]] = _line[4]
    return id2line

def load_conversations(file_name, id2line):
    conv_lines = open(file_name, encoding='iso-8859-1').read().split('\n')
    conversations = []
    for line in conv_lines[:-1]:
        _line = line.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
        conv = _line.split(',')
        for i in range(len(conv) - 1):
            input_line = id2line[conv[i]]
            target_line = id2line[conv[i + 1]]
            conversations.append((input_line, target_line))
    return conversations

movie_lines_file = "cornell movie-dialogs corpus/movie_lines.txt"
movie_conversations_file = "cornell movie-dialogs corpus/movie_conversations.txt"

id2line = load_movie_lines(movie_lines_file)
conversations = load_conversations(movie_conversations_file, id2line)

# Save to a DataFrame for easier manipulation
conversations_df = pd.DataFrame(conversations, columns=['input', 'response'])

# Save to a text file for the dataset
def save_to_text_file(conversations, file_name="conversations.txt"):
    with open(file_name, 'w') as f:
        for conv in conversations:
            f.write(conv[0] + '\n')
            f.write(conv[1] + '\n')

save_to_text_file(conversations)


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # You can also use "gpt2-medium", "gpt2-large", or "gpt2-xl"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )
    return dataset

def data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

train_dataset = load_dataset("conversations.txt", tokenizer)
data_collator = data_collator(tokenizer)

training_args = TrainingArguments(
    output_dir="./gpt2-conversational",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,2.814700
1000,2.758200
1500,2.731700
2000,2.719000
2500,2.770500
3000,2.730100
3500,2.699100
4000,2.687300
4500,2.716100
5000,2.666500


TrainOutput(global_step=26899, training_loss=2.6521611233006457, metrics={'train_runtime': 3229.1767, 'train_samples_per_second': 16.66, 'train_steps_per_second': 8.33, 'total_flos': 3514247184384000.0, 'train_loss': 2.6521611233006457, 'epoch': 1.0})

In [ ]:
# Save the model and tokenizer locally in Colab
model.save_pretrained('./gpt2-conversational')
tokenizer.save_pretrained('./gpt2-conversational')

('./gpt2-conversational/tokenizer_config.json',
 './gpt2-conversational/special_tokens_map.json',
 './gpt2-conversational/vocab.json',
 './gpt2-conversational/merges.txt',
 './gpt2-conversational/added_tokens.json')

In [ ]:
from google.colab import files

# Download the model files
files.download('./gpt2-conversational/config.json')
files.download('./gpt2-conversational/vocab.json')
files.download('./gpt2-conversational/merges.txt')
files.download('./gpt2-conversational/special_tokens_map.json')
files.download('./gpt2-conversational/tokenizer_config.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pre-trained GPT2 model and tokenizer
model_name = "gpt2"  # You can use a larger model like 'gpt2-medium' or 'gpt2-large'
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-conversational')
model = GPT2LMHeadModel.from_pretrained('./gpt2-conversational')

# Ensure the model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
def generate_response(prompt, model, tokenizer, max_length=40, num_return_sequences=1, temperature=0.3, top_k=20, top_p=0.85):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Adding a prefix to help the model differentiate between the user's input and the response
    prefix = "w"
    inputs = tokenizer.encode(prefix + prompt, return_tensors='pt').to(device)
    attention_mask = torch.ones(inputs.shape, device=device)

    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        attention_mask=attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        no_repeat_ngram_size=4,
        early_stopping=True
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Ensuring the response is limited to one sentence
    response = response.split('.')[0] + '.'
    return response

def chat(model, tokenizer):
    print("Start chatting with the model (type 'exit' to stop)!")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        response = generate_response(user_input, model, tokenizer)
        print("Bot:", response)

chat(model, tokenizer)

NameError: name 'model' is not defined

**max_length=150**: Increased maximum length to allow more extended responses.

**temperature=0.9:** Increased temperature to introduce more randomness and creativity.

**top_k=40:** Reduced top-k to make the model consider fewer options, thus making responses more focused.

**top_p=0.85**: Adjusted top-p for nucleus sampling to ensure cumulative probability covers a more focused range of tokens.

**no_repeat_ngram_size=3:** Increased n-gram repetition size to avoid repeating phrases.

https://chatgpt.com/c/ae4b0f8e-204e-4757-87be-60bc2d7a7bc8